### Rede de insteresses dos deputados (Tema Aborto)

#### OBJETIVO

---

Entender quem são os atores centrais na Câmara na discussão sobre o *aborto*.

#### METODOLOGIA

---

A rede consiste num multigrafo $G = \{V,E\}$, no qual:

* $V = \{d_{i}\}$, é o conjunto dos deputados $d_{i}$.
* $E = \{\{d_{i}, d_{j}\}\}$, é o conjunto das relações entre deputados. 
Cada aresta representa uma proposição em que ambos os deputados atuaram, ou seja, estão presentes no despacho das tramitações dessa proposição.

O caminho entre os nós pode ser entendido como o grau de "proximidade" da discussão na qual dois deputados estão envolvidos, supondo que os deputados atuam em proposições que condizem com seus próprios interesses.

Calculamos a intermediação (*betweness*) da rede para determinar quem mais influencia nas pautas sobre o tema.

Para cada deputado, sua intermediação consiste na razão do número de menores caminho que passam por ele pelo total, ou seja:

$g(d_{i}) = \sum_{j\neq k\neq i}\frac{\sigma_{d_{j}d_{k}}(d_{i})}{\sigma_{d_{j}d_{k}}} = 
\frac{2 \sum_{j\neq k\neq i}{\sigma_{d_{j}d_{k}(d_{i})}}}{n(n-1)}$

onde $\sigma_{jk}$ é o menor caminho do nó $d_{j}$ ao $d_{k}$.


#### Referências

---
[Intermediação - Wikipédia](https://pt.wikipedia.org/wiki/Intermedia%C3%A7%C3%A3o)

2. Ver se existem grupos específicos que dominam essas pautas (através do coeficiente de agrupamento local).

In [3]:
import networkx as nx
import yaml

# DATA ANALYSIS & VIZ TOOLS
from copy import deepcopy
import pandas as pd

pd.options.display.max_columns = 999

import plotly
import plotly.graph_objs as go
import plotly.offline as offline

offline.init_notebook_mode(connected=True)
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 12)

# FILES STORAGE
import os
cwd = os.getcwd()
RAW_DATA = cwd+'/raw_data/' # put primary data here
DATA = cwd+'/data/' # put processed data here

# BASIC INFOS
from datetime import datetime as dt

# LEGISLATURAS
i = round((dt.now().year - 2003)/4+0.5)
period = list(range(0,i))
legislaturas = {}
for x in period:
    
    t = ('{}-01-31'.format(2003+4*x), '{}-02-01'.format(2007+4*x))
    legislaturas[t] = 52+x

Populating the interactive namespace from numpy and matplotlib


## Bases de dados

### Parlamentares
A base `deputados` contém as informações dos parlamentares desde a 52ª legislatura.

In [4]:
deputados = pd.read_csv(RAW_DATA+'deputados.csv', index_col=0)
print(deputados.info())
deputados.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2481 entries, 0 to 2480
Data columns (total 16 columns):
id                      2481 non-null int64
capnum                  2481 non-null int64
datacaptura             2481 non-null object
idecadastro             2481 non-null int64
numlegislatura          2481 non-null int64
nomeparlamentar         2481 non-null object
sexo                    2481 non-null object
profissao               2131 non-null object
legendapartidoeleito    2481 non-null object
ufeleito                2481 non-null object
condicao                2481 non-null object
situacaomandato         2481 non-null object
matricula               2481 non-null int64
gabinete                2481 non-null int64
anexo                   2481 non-null int64
fone                    513 non-null object
dtypes: int64(7), object(9)
memory usage: 329.5+ KB
None


,id,capnum,datacaptura,idecadastro,numlegislatura,nomeparlamentar,sexo,profissao,legendapartidoeleito,ufeleito,condicao,situacaomandato,matricula,gabinete,anexo,fone
0,2211,2,2015-11-02 13:14:39,141542,55,SÉRGIO MORAES,M,COMERCIANTE,PTB,RS,Titular,Em Exercício,512,258,4,3215-5258
1,2215,2,2015-11-02 13:14:39,178874,55,SERGIO VIDIGAL,M,NaN,PDT,ES,Titular,Em Exercício,283,812,4,3215-5812


### Proposições sobre aborto

A base `props_aborto` contém a descrição das proposições sobre aborto já apresentadas na Câmara.

In [5]:
props_aborto = pd.read_csv(RAW_DATA+'props_aborto_teor.csv')
print(props_aborto.info())
props_aborto.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 15 columns):
Proposições             68 non-null object
Classe                  68 non-null object
Apreciação              68 non-null object
Regime de Tramitação    68 non-null object
Natureza                68 non-null object
Ementa                  68 non-null object
Explicação              38 non-null object
Autor                   68 non-null object
UF                      68 non-null object
Partido                 68 non-null object
Apresentação            68 non-null object
Ano                     68 non-null int64
Unnamed: 12             55 non-null object
Situação                68 non-null object
Link                    68 non-null object
dtypes: int64(1), object(14)
memory usage: 8.0+ KB
None


,Proposições,Classe,Apreciação,Regime de Tramitação,Natureza,Ementa,Explicação,Autor,UF,Partido,Apresentação,Ano,Unnamed: 12,Situação,Link
0,PL 7122/2017,Aborto,Plenário,"Prioridade (151, II, RICD)",A favor,Dispõe sobre a concessão de licença e de salár...,"Altera o Decreto-Lei nº 5.452, de 1943 e a Lei...",Luciano Ducci,PR,PSB,3/15/2017,2017,Apensada,Aguardando Criação de Comissão Temporária pela...,http://www2.camara.gov.br/proposicoesWeb/ficha...
1,PDC 1487/2014,Aborto,Plenário,"Ordinária (151, III, RICD)",Contra,"Susta a eficácia da Portaria nº 415, de 21 de ...",NaN,Eduardo Cunha,RJ,PMDB,5/27/2014,2014,Raiz,Pronta para Pauta,http://www2.camara.gov.br/proposicoesWeb/ficha...


In [6]:
props_aborto['Apresentação'] = pd.to_datetime(props_aborto['Apresentação'])

In [7]:
print(props_aborto['Apresentação'].min())
print(props_aborto['Apresentação'].max())

1977-11-10 00:00:00
2017-03-15 00:00:00


In [8]:
props_aborto['Proposições'].values

array(['PL 7122/2017', 'PDC 1487/2014', 'PDC 1490/2014', 'PDC 1757/2005',
       'PDC 1832/2005', 'PDC 2840/2010', 'PDC 42/2007', 'PDC 467/2000',
       'PDC 565/2012', 'PDC 566/2012', 'PDC 737/1998', 'PEC 164/2012',
       'PEC 25/1995', 'PEC 303/2008', 'PEC 408/2005', 'PEC 553/2002',
       'PEC 571/2002', 'PEC 62/2003', 'PL 1008/1988', 'PL 1035/1991',
       'PL 1085/2011', 'PL 1091/2003', 'PL 1097/1991', 'PL 1104/1991',
       'PL 1107/1991', 'PL 1135/1991', 'PL 1190/2011', 'PL 1459/2003',
       'PL 1522/2015', 'PL 1545/2011', 'PL 176/1995', 'PL 1763/2007',
       'PL 20/1991', 'PL 2059/2003', 'PL 2155/2007', 'PL 2423/1989',
       'PL 2433/2007', 'PL 2438/1989', 'PL 244/1983', 'PL 2690/2007',
       'PL 2852/2000', 'PL 3039/2015', 'PL 3166/1992', 'PL 3207/2008',
       'PL 3348/2000', 'PL 3609/1993', 'PL 3673/2008', 'PL 3748/2008',
       'PL 3783/2008', 'PL 3872/1989', 'PL 3983/2015', 'PL 4396/2016',
       'PL 4403/2004', 'PL 4474/1977', 'PL 4641/2016', 'PL 4642/2016',
       '

## Tramitações

A base `tramitacoes_aborto` contém as tramitações referentes às proposições acima. 

In [9]:
tramitacoes_aborto = pd.read_csv(RAW_DATA+'tramitacoes_aborto.csv', index_col=0)
print(tramitacoes_aborto.info())
tramitacoes_aborto.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254 entries, 0 to 1253
Data columns (total 25 columns):
level_0                                    1254 non-null int64
index                                      1254 non-null int64
idProposicao                               1254 non-null int64
ementa                                     1254 non-null object
situacao                                   1254 non-null object
numero                                     1254 non-null int64
ano                                        1254 non-null int64
tipo                                       1254 non-null object
ultimaAcao_tramitacao_codOrgao             1254 non-null int64
ultimaAcao_tramitacao_orgao                1254 non-null object
ultimaAcao_tramitacao_descricao            1254 non-null object
ultimaAcao_tramitacao_data                 1254 non-null object
ultimaAcao_tramitacao_ordemDeTramitacao    1254 non-null int64
andamento_tramitacao_codOrgao              1254 non-null int64
andam

,level_0,index,idProposicao,ementa,situacao,numero,ano,tipo,ultimaAcao_tramitacao_codOrgao,ultimaAcao_tramitacao_orgao,ultimaAcao_tramitacao_descricao,ultimaAcao_tramitacao_data,ultimaAcao_tramitacao_ordemDeTramitacao,andamento_tramitacao_codOrgao,andamento_tramitacao_Orgao,andamento_tramitacao_descricao,andamento_tramitacao_inteiroTeor,andamento_tramitacao_codReuniao,andamento_tramitacao_data,andamento_tramitacao_ordemDeTramitacao,data_captura,unique_id,ordem_tramitacao,next_id,previous_id
0,0,97,484558,Susta os efeitos gerados pelo ato do Poder Exe...,PLEN - Devolvida ao Autor,2840,2010,PDC,180,PLEN - PLENÁRIO,"Devolvido ao Autor, nos termos do § 1º do arti...",2010-11-04 00:00:00,14,4,MESA,Encerramento automático do Prazo de Recurso. N...,NaN,NaN,2010-11-03 00:00:00,13,2018-03-22 20:36:41,-3406100917335070313,3,2.575394e+18,5.976673e+18
1,112,97,484558,Susta os efeitos gerados pelo ato do Poder Exe...,PLEN - Devolvida ao Autor,2840,2010,PDC,180,PLEN - PLENÁRIO,"Devolvido ao Autor, nos termos do § 1º do arti...",2010-11-04 00:00:00,14,180,PLEN - PLENÁRIO,"Devolvido ao Autor, nos termos do § 1º do arti...",NaN,NaN,2010-11-04 00:00:00,14,2018-03-22 20:36:41,2575394321104112753,4,NaN,-3.406101e+18


In [10]:
tramitacoes_aborto['nomeProposicao'] = tramitacoes_aborto['tipo'].map(str) + ' ' + tramitacoes_aborto['numero'].map(str) + '/' + tramitacoes_aborto['ano'].map(str)

In [11]:
tramitacoes_aborto['numlegislatura'] = np.nan

for idx, row in tramitacoes_aborto.iterrows():
    
    for periodo in legislaturas.keys():
        
        inicio = periodo[0]
        fim = periodo[1]
        
        if (row['andamento_tramitacao_data'] >= inicio) and (row['andamento_tramitacao_data'] <= fim):

            tramitacoes_aborto.loc[idx,['numlegislatura']] = legislaturas[periodo]
            pass

In [12]:
tramitacoes_aborto = tramitacoes_aborto.loc[:,['idProposicao',
                                               'nomeProposicao',
                                               'andamento_tramitacao_descricao', 
                                               'andamento_tramitacao_data', 
                                               'unique_id',
                                               'numlegislatura']]
tramitacoes_aborto.head()

,idProposicao,nomeProposicao,andamento_tramitacao_descricao,andamento_tramitacao_data,unique_id,numlegislatura
0,484558,PDC 2840/2010,Encerramento automático do Prazo de Recurso. N...,2010-11-03 00:00:00,-3406100917335070313,53.0
1,484558,PDC 2840/2010,"Devolvido ao Autor, nos termos do § 1º do arti...",2010-11-04 00:00:00,2575394321104112753,53.0
2,484558,PDC 2840/2010,Apresentação do Projeto de Decreto Legislativo...,2010-08-04 00:00:00,7706943970270103526,53.0
3,484558,PDC 2840/2010,"Devolva-se a proposição, por contrariar o disp...",2010-08-17 00:00:00,8182031166867923856,53.0
4,484558,PDC 2840/2010,Prazo de Recurso (5 sessões ordinárias a parti...,2010-08-17 00:00:00,5976673334173288414,53.0


In [86]:
tramitacoes_aborto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254 entries, 0 to 1253
Data columns (total 6 columns):
idProposicao                      1254 non-null int64
nomeProposicao                    1254 non-null object
andamento_tramitacao_descricao    1254 non-null object
andamento_tramitacao_data         1254 non-null object
unique_id                         1254 non-null int64
numlegislatura                    981 non-null float64
dtypes: float64(1), int64(2), object(3)
memory usage: 68.6+ KB


## Rede *(grafo)*

### Conjunto de nós

In [13]:
deps_legs = deputados[deputados['condicao'] == 'Titular'][['numlegislatura', 'nomeparlamentar']]\
                .drop_duplicates()\
                .groupby('numlegislatura')['nomeparlamentar']\
                .apply(lambda x: x.tolist())\
                .to_dict()
deps_legs[54][:5]

['BOHN GASS',
 'FLÁVIA MORAIS',
 'ANDREIA ZITO',
 'MARLLOS SAMPAIO',
 'OZIEL OLIVEIRA']

In [93]:
len(deps_legs[52])

513

### Conjunto de arestas

In [14]:
tramitacoes_deps = pd.DataFrame(columns=['deputado', 'idProposicao', 'numlegislatura'])

for leg in legislaturas.values():
    
    for dep in deps_legs[leg]:
        
        t = tramitacoes_aborto[tramitacoes_aborto['andamento_tramitacao_descricao']\
                               .str.upper()\
                               .str.contains(dep.upper())]
        if len(t) > 0:
            tram_dep = deepcopy(t)
            tram_dep['deputado'] = dep
            tramitacoes_deps = tramitacoes_deps.append(tram_dep[['deputado', 
                                                                 'idProposicao', 
                                                                 'numlegislatura']])

In [94]:
tramitacoes_deps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1331 entries, 904 to 435
Data columns (total 3 columns):
deputado          1331 non-null object
idProposicao      1331 non-null float64
numlegislatura    1189 non-null float64
dtypes: float64(2), object(1)
memory usage: 41.6+ KB


In [95]:
tramitacoes_deps = tramitacoes_deps.dropna(subset=['numlegislatura']) # Desconsiderando relações em que o exercício do deputado é desconhecido
tramitacoes_deps = tramitacoes_deps.drop_duplicates().reset_index(drop=True) # Desconsiderando relação de um deputado com ele mesmo
tramitacoes_deps.head()

,deputado,idProposicao,numlegislatura
0,IRINY LOPES,281681.0,52.0
1,EDNA MACEDO,18716.0,52.0
2,DRA. CLAIR,18716.0,52.0
3,LUIZ BASSUMA,16299.0,53.0
4,LUIZ BASSUMA,345103.0,53.0


In [100]:
relacoes = pd.merge(tramitacoes_deps, tramitacoes_deps, on=['numlegislatura', 'idProposicao'])

### remover combinações repetidas:
# 
# -> Tá dando erro no cálculo do degree. Quando a aresta é de um vértice para ele mesmo, 
# o grau está sendo contabilizado duas vezes.
#
# relacoes.loc[:,'set'] = relacoes.apply(lambda x: str(set([x.deputado_x,x.deputado_y])),axis=1)
# relacoes = relacoes.drop_duplicates(subset=['set','numlegislatura','idProposicao'])

relacoes.head()

,deputado_x,idProposicao,numlegislatura,deputado_y
0,IRINY LOPES,281681.0,52.0,IRINY LOPES
1,IRINY LOPES,281681.0,52.0,OSMÂNIO PEREIRA
2,IRINY LOPES,281681.0,52.0,NILTON BAIANO
3,OSMÂNIO PEREIRA,281681.0,52.0,IRINY LOPES
4,OSMÂNIO PEREIRA,281681.0,52.0,OSMÂNIO PEREIRA


In [102]:
relacoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5332 entries, 0 to 5331
Data columns (total 4 columns):
deputado_x        5332 non-null object
idProposicao      5332 non-null float64
numlegislatura    5332 non-null float64
deputado_y        5332 non-null object
dtypes: float64(2), object(2)
memory usage: 208.3+ KB


In [103]:
print('Número de arestas na rede: ', len(relacoes))
relacoes[['numlegislatura','idProposicao']].groupby('numlegislatura').count()

Número de arestas na rede:  5332


,idProposicao
numlegislatura,
52.0,248
53.0,3585
54.0,216
55.0,1283


### Representação Gráfica

In [104]:
def create_viz_network(G):

    # create node trace:
    node_trace = go.Scatter(x = [], y = [], 
                            text = [], mode = 'markers',
                            hoverinfo='text',
                            marker = dict(
                                showscale = True,
                                colorscale = 'YIGnBu',
                                reversescale = True,
                                color = [],
                                size = 10,
                                colorbar = dict(
                                    thickness = 15,
                                    title = 'Número de conexões',
                                    xanchor = 'left',
                                    titleside = 'right'),
                                line = dict(width = 1)))

    # set node positions
    pos = nx.spring_layout(G)
    for node in G.nodes():
        G.node[node]['pos']= pos[node]

    for node in G.nodes():
        x, y = G.node[node]['pos']
        node_trace['x'].append(x)
        node_trace['y'].append(y)

    # create edge trace:
    edge_trace = go.Scatter(x = [], y = [], text = [],
                         line = dict(width = 1, color = '#888'),
                         mode = 'lines')

    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]
        #edge_trace['text'] += str(matrix.loc[edge[0], edge[1]])[:5]

    for node in list(G.nodes()):
        
        degree = G.degree(node)/2
        
        node_trace['marker']['color'].append(degree)
        node_info = '# de conexões: '+str(degree)
        node_trace['text'].append(node + '<br>' + node_info)

    layout = go.Layout(
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
    
    fig = go.Figure(data=[edge_trace, node_trace], layout=layout)
    #offline.iplot(fig, filename='networkx')
    
    graus = [(node, G.degree(node)/2) for node in list(G.nodes())]
    graus.sort(key=lambda x: x[1], reverse=True)
    
    return fig, pd.DataFrame(graus, columns=['Deputado(a)', 'Grau']).set_index('Deputado(a)')

In [105]:
offline.iplot(create_viz_network(nx.from_pandas_edgelist(
    relacoes[relacoes['numlegislatura'] == 54], 
    source = 'deputado_x', 
    target = 'deputado_y',
    create_using = nx.MultiGraph()
))[0])

In [106]:
import networkx as nx
Graphs = {}
degrees = pd.DataFrame(index=['Deputado(a)'])
for leg in [52,53,54,55]:
    G = nx.from_pandas_edgelist(
        relacoes[relacoes['numlegislatura'] == leg], 
        source = 'deputado_x', 
        target = 'deputado_y',
        create_using = nx.MultiGraph()
    )
    
    Graphs[leg] = G
    
    print('\nLegislatura:', leg, 
          '| Nº de nós:', len(G.nodes), 
          '| Nº de arestas:', len(G.edges))
           
    degrees = degrees.merge(pd.DataFrame(create_viz_network(G)[1]).rename(columns={'Grau':leg}),left_index=True,right_index=True,how='outer')
degrees.head()


Legislatura: 52 | Nº de nós: 32 | Nº de arestas: 248

Legislatura: 53 | Nº de nós: 74 | Nº de arestas: 3585

Legislatura: 54 | Nº de nós: 33 | Nº de arestas: 216

Legislatura: 55 | Nº de nós: 47 | Nº de arestas: 1283


,52,53,54,55
AFONSO FLORENCE,NaN,NaN,12.0,NaN
ALBERTO FILHO,NaN,NaN,2.0,NaN
ALCENI GUERRA,NaN,5.0,NaN,NaN
ALCEU MOREIRA,NaN,NaN,NaN,35.0
ALINE CORRÊA,NaN,55.0,NaN,NaN


## Intermediação

In [107]:
import operator

betweenness = pd.DataFrame(index=['Deputado(a)'])
for leg in [52,53,54,55]:
    G = Graphs[leg]
    bet = nx.betweenness_centrality(Graphs[leg])
    bet = sorted(bet.items(), key=operator.itemgetter(1), reverse=1)
    df = pd.DataFrame.from_records(bet).rename(columns={0:'Deputado(a)',1:leg}).set_index('Deputado(a)')
    betweenness = betweenness.merge(df,left_index=True,right_index=True,how='outer')

In [108]:
betweenness.sort_values(55,ascending=0).head()

,52,53,54,55
FLAVINHO,NaN,NaN,NaN,0.109179
GIVALDO CARIMBÃO,NaN,0.002711,0.0,0.037681
JOÃO CAMPOS,NaN,0.013318,0.0,0.035749
MARCOS ROGÉRIO,NaN,NaN,NaN,0.035749
ALCEU MOREIRA,NaN,NaN,NaN,0.000000


In [112]:
def informations_by_id(leg):
    
    ### Graph
    offline.iplot(create_viz_network(Graphs[leg])[0])
    
    ### Degree
    degrees_leg = degrees.loc[:,[leg]].sort_values(leg,ascending=False)
    
    ### Betweenness
    betweenness_leg = betweenness.loc[:,[leg]].sort_values(leg,ascending=False)
    
    return degrees_leg, betweenness_leg

In [113]:
result = informations_by_id(55)

In [114]:
print('Parlamentares mais relacionados ao tema')
result[0].head(20)

Parlamentares mais relacionados ao tema


,55
FLAVINHO,41.0
JOÃO CAMPOS,39.0
GIVALDO CARIMBÃO,39.0
MARCOS ROGÉRIO,39.0
RUBENS PEREIRA JÚNIOR,38.0
ERIKA KOKAY,37.0
JANDIRA FEGHALI,37.0
PASTOR EURICO,37.0
LUCIANO DUCCI,37.0
PR. MARCO FELICIANO,36.0


In [115]:
print('Parlamentares mais influentes no tema')
result[1].head(20)

Parlamentares mais influentes no tema


,55
FLAVINHO,0.109179
GIVALDO CARIMBÃO,0.037681
JOÃO CAMPOS,0.035749
MARCOS ROGÉRIO,0.035749
ALCEU MOREIRA,0.000000
IVAN VALENTE,0.000000
LEONARDO PICCIANI,0.000000
LUCIANO DUCCI,0.000000
LUIZ COUTO,0.000000
MARIA DO ROSÁRIO,0.000000
